In [2]:
import pandas as pd
import os

#Dataframe Visual Settings
pd.set_option('display.max_rows',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',400)

In [4]:
def clean_bank_statement_file(filename):
    """
    Extract, modify & clean bank statement raw file for neater format
    """
    #Cleaning data
    # csv_file = '\Bank Statements\'' + filename
    df = pd.read_csv(filename,
                     usecols=['Transaction Number','Date','Memo','Amount Debit','Amount Credit'],
                     skiprows=3)
    
    df.fillna({'Amount Debit':0, 'Amount Credit':0},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
    df.sort_values(by=['Date'],ascending=False,ignore_index=True,inplace=True)
    df['Memo'].fillna('Default',inplace=True)
    df['Transaction Number'] = df['Transaction Number'].apply(lambda x: x.split('**')[1])
    df['Purchase Date'] = df[df['Memo'].str.contains('1574 ',na=False)]['Memo']\
        .apply(lambda x: x.split('1574 ')[1])
    df['Purchase Date'] = pd.to_datetime(df['Purchase Date'],format = '%m/%d %H:%M')
    df['Purchase Date'] = df['Purchase Date']+pd.DateOffset(years=120)
    df.loc[df['Purchase Date'].isna(),'Purchase Date'] = df['Date'].copy()
    df['Memo'] = df['Memo'].apply(lambda x: x.split('*****')[0])
    df['Content'] = df['Transaction Number'] + ' ' + df['Memo']
    df['Amount'] = df['Amount Debit'].astype(float) + df['Amount Credit'].astype(float)
    df['Category'] = 'Default'
    df.drop(df[df['Content'].str.contains("INTERNET TRANSFER",na=False)].index,inplace=True)
    df.reset_index(drop=True,inplace=True)
    df.drop(['Transaction Number','Memo','Amount Debit','Amount Credit'],
            axis=1,inplace=True)
    df.sort_values(by=['Purchase Date'],ascending=False, inplace=True)
    return df

In [7]:
def categorize(df):
    #Easier Categorization
    df.loc[df['Content'].str.contains("UBER EATS",na=False),'Category'] = 'Dining Out'
    df.loc[df['Content'].str.contains("HARRIS",na=False),'Category'] = 'Groceries'
    df.loc[df['Content'].str.contains("GIANT",na=False),'Category'] = 'Groceries'
    df.loc[df['Content'].str.contains("USAA",na=False),'Category'] = 'USAA Insurance'
    df.loc[df['Content'].str.contains("Accenture",na=False),'Category'] = 'Pay Check'
    df.loc[df['Content'].str.contains("XSPORT",na=False),'Category'] = 'Gym'
    df.loc[df['Content'].str.contains("DISTRICT MARTIAL ARTS",na=False),'Category'] = 'Gym'
    df.loc[df['Content'].str.contains("PARKING",na=False),'Category'] = 'Tolls/Uber/Metro/Parking'
    df.loc[df['Content'].str.contains("NAZRET",na=False),'Category'] = 'Dining Out'
    df.loc[df['Content'].str.contains("TAJ OF INDIA",na=False),'Category'] = 'Dining Out'
    df.loc[df['Content'].str.contains("DCPILLAR",na=False),'Category'] = 'Tithe'
    df.loc[df['Content'].str.contains("GOOGLE",na=False),'Category'] = 'Entertainment'
    df.loc[df['Content'].str.contains("VENMO/CASHOUT",na=False),'Category'] = 'Venmo Extra'
    df.loc[df['Content'].str.contains("CITGO",na=False),'Category'] = 'Gas'
    df.loc[df['Content'].str.contains("SHELL",na=False),'Category'] = 'Gas'
    df.loc[df['Content'].str.contains("PUPATELLA",na=False),'Category'] = 'Dining Out'
    df.loc[df['Content'].str.contains("GOOD COMPANY DONUT",na=False),'Category'] = 'Dining Out'
    df.loc[df['Content'].str.contains("STARBUCKS",na=False),'Category'] = 'Dining Out'
    df.loc[df['Content'].str.contains("UBER TRIP",na=False),'Category'] = 'Tolls/Uber/Metro/Parking'
    df.loc[df['Content'].str.contains("VERIZON",na=False),'Category'] = 'Utilities'
    df.loc[df['Content'].str.contains("WASHINGTON GAS",na=False),'Category'] = 'Utilities'
    df.loc[df['Content'].str.contains("ENERGY",na=False),'Category'] = 'Utilities'
    df.loc[df['Content'].str.contains("TOM COLEMAN",na=False),'Category'] = 'Phone'
    df.loc[df['Content'].str.contains("STDNT LOAN",na=False),'Category'] = 'Student Loans'
    df.loc[(df['Content'].str.contains("VENMO/PAYMENTWALTER COLEMAN Default",na=False)) &
           (df['Amount'] == -668.75),'Category'] = 'Rent'
    df.loc[df['Content'].str.contains("Margaret Coleman",na=False),'Category'] = 'Extra'
    df.loc[df['Content'].str.contains("Person-to-Person TransferPAYPAL",na=False),'Category'] = 'Extra'
    return df

In [5]:
df = clean_bank_statement_file('DecChecking23.csv')

In [8]:
df = categorize(df)

In [9]:
df

,Date,Purchase Date,Content,Amount,Category
0,2020-12-23,2020-12-23 00:00:00,USAA PC/PAYMENTWALTER COLEMAN Default,-187.68,USAA Insurance
2,2020-12-23,2020-12-23 00:00:00,VENMO/PAYMENTWALTER COLEMAN Default,-845.00,Default
6,2020-12-23,2020-12-23 00:00:00,VENMO/PAYMENTWALTER COLEMAN Default,-14.00,Default
1,2020-12-23,2020-12-23 00:00:00,Pillar DC/Pillar DCPILLAR CHURCH OF WASHI,-310.97,Tithe
3,2020-12-23,2020-12-22 19:39:00,POS PURCHASE FEEITA FEES PAYPAL *MYPROTEIN 35314369001GB 000000,-1.06,Default
4,2020-12-23,2020-12-22 19:39:00,POS PURCHASE Non-PINPAYPAL *MYPROTEIN 35314369001 GB000000,-35.33,Default
5,2020-12-23,2020-12-22 17:57:00,POS PURCHASE Non-PINAMZN Mktp US*MW02D84K3 Amzn.com/bill WAIN7300,-10.38,Default
9,2020-12-22,2020-12-21 23:29:00,POS PURCHASE Non-PINAMZN Mktp US*AV60Z0NN3 Amzn.com/bill WAIN0002,-44.51,Default
7,2020-12-22,2020-12-21 22:13:00,POS PURCHASE Non-PINPAYPAL *UDEMY 888-838-5432 CA IN1090,-19.99,Default
21,2020-12-21,2020-12-21 09:09:00,POS PURCHASE Non-PINTST* GOOD COMPANY DONUT ARLINGTON VAIN0700,-15.73,Dining Out


In [11]:
df.groupby(df['Category']).sum()

,Amount
Category,
Default,-1625.66
Dining Out,-173.18
Entertainment,-4.99
Gas,-63.45
Groceries,-242.79
Gym,-180.95
Pay Check,4157.65
Tithe,-615.68
Tolls/Uber/Metro/Parking,-14.58


In [14]:
df[df['Category'] == 'Default'][['Content','Amount']].sort_values(by=['Amount'])

,Content,Amount
2,VENMO/PAYMENTWALTER COLEMAN Default,-845.00
41,POS PURCHASE Non-PINsenseofplacecafe.com 972-543190919 CAIN7900,-159.69
43,Washington Gas L/8449274427Walter Coleman,-101.34
22,VENMO/PAYMENTWALTER COLEMAN Default,-53.18
30,VENMO/PAYMENTWALTER COLEMAN Default,-48.00
9,POS PURCHASE Non-PINAMZN Mktp US*AV60Z0NN3 Amzn.com/bill WAIN0002,-44.51
62,VENMO/PAYMENTWALTER COLEMAN Default,-37.50
4,POS PURCHASE Non-PINPAYPAL *MYPROTEIN 35314369001 GB000000,-35.33
38,POS PURCHASE Non-PINDEL RIO RESTAURANT LEESBURG VA032716,-32.83
31,POS PURCHASE Non-PINAMAZON.COM*HT2A90Q83 AM AMZN.COM/BILL WAIN9000,-31.80
